In [1]:
import tensorflow 
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D,MaxPooling2D, LeakyReLU, PReLU , Reshape

In [2]:
training_filename = "C:\\Users\\F. Oseni\\jupyter notebook\\MTCNN-Tensorflow-master1\\MTCNN-Tensorflow-master1\\validation\\12\\train_PNet_landmark.tfrecord_shuffle.000"
validation_filename = "C:\\Users\\F. Oseni\\jupyter notebook\\MTCNN-Tensorflow-master1\\MTCNN-Tensorflow-master1\\validation\\12\\train_PNet_landmark.tfrecord_shuffle.001"

In [3]:
BATCH_SIZE = 384
buffer_size=10 * BATCH_SIZE

In [4]:
def read_record(example):
    image_features = tf.io.parse_single_example(
       example,
       features={
           'image/encoded': tf.io.FixedLenFeature([], tf.string),#one image  one record
           'image/label': tf.io.FixedLenFeature([], tf.int64),
           'image/roi': tf.io.FixedLenFeature([4], tf.float32),
           'image/landmark': tf.io.FixedLenFeature([10],tf.float32)
       }
    )
    image = tf.io.decode_raw(image_features['image/encoded'], tf.uint8)
    image = tf.reshape(image, [12, 12, 3])
    image = (tf.cast(image, tf.float32)-127.5) / 128
    
    # image = tf.image.per_image_standardization(image)
    label = tf.cast(image_features['image/label'], tf.float32)
    roi = tf.cast(image_features['image/roi'],tf.float32)
    landmark = tf.cast(image_features['image/landmark'],tf.float32)
    return image, (label, roi)

In [5]:
def get_batch(filenames):
    option_no_order = tf.data.Options()
    option_no_order.experimental_deterministic = False
    
    
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.with_options(option_no_order)
    
    dataset = dataset.interleave(tf.data.TFRecordDataset)
    dataset = dataset.map(read_record)
    
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    
    dataset = dataset.prefetch(buffer_size=buffer_size)
    
    return dataset

In [6]:
def get_training_datest():
    return get_batch(training_filename)

def get_validation_dataset():
    return get_batch(validation_filename)

In [ ]:
import keras.backend as K
def euclidean_distance_loss(y_true, y_pred):
    # Euclidean distance as a measure of loss (Loss function) 
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [15]:
#splitting the tfrecords into training and validaton tfrecords
import tensorflow as tf
tfrecord_path = "C:\\Users\\F. Oseni\\jupyter notebook\\MTCNN-Tensorflow-master1\\MTCNN-Tensorflow-master1\\validation\\12\\train_PNet_landmark.tfrecord_shuffle.000"
split_size = 1000000
def split_tfrecord(tfrecord_path, split_size):
    with tf.Graph().as_default(), tf.Session() as sess:
        ds = tf.data.TFRecordDataset(tfrecord_path).batch(split_size)
        batch = ds.make_one_shot_iterator().get_next()
        part_num = 0
        while True:
            try:
                records = sess.run(batch)
                part_path = tfrecord_path + '.{:03d}'.format(part_num)
                with tf.python_io.TFRecordWriter(part_path) as writer:
                    for record in records:
                        writer.write(record)
                part_num += 1
            except tf.errors.OutOfRangeError: break

In [16]:
split_tfrecord(tfrecord_path, split_size)

In [13]:
def p_net():
      
        inputs = tf.keras.Input(shape=(12,12,3))

        model = tf.keras.layers.Conv2D(10,(3, 3), strides=1, name='conv1',padding='SAME')(inputs)
        model = tf.keras.layers.PReLU(name='prelu1')(model)
        model = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2))(model)

        model = tf.keras.layers.Conv2D(16,(3, 3), strides=1,name='conv2')(model)
        model = tf.keras.layers.PReLU(name='prelu2')(model)

        model = tf.keras.layers.Conv2D(32,(3,3),strides=1,name='conv3')(model)
        model = tf.keras.layers.PReLU(name='prelu3')(model)

        classifier1 = tf.keras.layers.Conv2D(2, (1, 1), activation='softmax',name='classifier1')(model)
        #I am not sure if this line of code is impoertant
        #classifier1 = tf.keras.layers.Reshape((2,))(classifier1)   # this layer has to be deleted in order to enalbe arbitraty shape input

        bbox_regress = Conv2D(4, (1, 1),name='bbox1')(model)
        #I am not sure if this line of code is impoertant
        #bbox_regress = tf.keras.layers.Reshape((4,))(bbox_regress)

        #THis last code is not usually included in most mtcnn replicas
        #landmark = Conv2D(10, (1, 1),name='landmark1')(model)
        #landmark =  tf.keras.layers.Reshape((10,))(landmark)

        model = tf.keras.Model(inputs=inputs, outputs=[classifier1,bbox_regress])
        model.compile(optimizer=tf.optimizers.Adam(0.01), loss=['binary_crossentropy','mean_squared_error'], metrics=['accuracy'])

        return model

In [14]:
modelsss = p_net()

In [ ]:
modelsss.fit(get_training_datest(), steps_per_epoch=100, epochs=30, validation_data=get_validation_dataset(), validation_steps=50)

In [ ]:
model.summary()

In [52]:
def p_net():

    model = Sequential() #there are 2 types of models but this is the most common
    model.add(Conv2D(10,(3, 3), strides=1, name='conv1',padding='SAME', input_shape =(12, 12, 3) ) )
    model.add(PReLU(name='prelu1'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

    model.add(Conv2D(16,(3, 3), strides=1,name='conv2', input_shape =(5, 5, 10)))
    model.add(PReLU(name='prelu2'))

    model.add(Conv2D(32,(3,3),strides=1,name='conv3'))
    model.add(PReLU(name='prelu3'))

    model.add(Conv2D(2, (1, 1), activation='softmax',name='classifier1'))
    model.add(PReLU(name='prelu4'))

    model.add(Dense(4,name='bbox_regression'))

    model.add(Dense(10,name='landmark'))

    #model = tf.keras.Model(inputs=, outputs=)
    model.compile(optimizer=tf.optimizers.Adam(0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

    #my_adam = adam(lr = 0.00001)




#R-Net

def r_net():
    model = Sequential() #there are 2 types of models but this is the most common
    model.add(Conv2D(28, (3, 3), strides=1, name='conv1', input_shape =(24, 24, 3)))
    model.add(PReLU(name='prelu1'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

    model.add(Conv2D(48, (3, 3), strides=1, name='conv2'))
    model.add(PReLU(name='prelu2'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

    model.add(Conv2D(64, (2, 2), strides=1, name='conv3'))
    model.add(PReLU(name='prelu3'))

    model.add(Dense(128))
    model.add(PReLU(name='prelu4'))

    model.add(Dense(2))
    model.add(Activation('softmax'))

    model.add(Dense(4))#THere is something being used input_layer_name='prelu4)

#O-Net

def o_net():
    model = Sequential() #there are 2 types of models but this is the most common
    model.add(Conv2D(32, (3, 3), strides=1, name='conv1'), input_shape =(48, 48, 3))
    model.add(PReLU(name='prelu1'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

    model.add(Conv2D(64, (3, 3), strides=1, name='conv2'))
    model.add(PReLU(name='prelu2'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

    model.add(Conv2D(64, (3, 3), strides=1, name='conv3'))
    model.add(PReLU(name='prelu3'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

    model.add(Conv2D(128, (2, 2), strides=1, name='conv4'))
    model.add(PReLU(name='prelu4'))

    model.add(Dense(256))
    model.add(PReLU(name='prelu5'))

    model.add(Dense(2))
    model.add(Activation('softmax'))

    model.add(Dense(4))#THere is something being used input_layer_name='prelu5

    model.add(Dense(10))#THere is something being used input_layer_name='prelu5